# Quantum Magnetic Navigation API Demo

This notebook demonstrates how to use the Quantum Magnetic Navigation API to estimate positions based on magnetic field measurements.

## Setup

First, we need to make sure the API server is running. You can start it with:

```bash
uvicorn qmag_nav.service.api:app --reload
```

Then we can interact with it using the requests library.

In [ ]:
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# Set the base URL for the API
BASE_URL = "http://localhost:8000"

## Check API Health

Let's first check if the API is running by calling the health endpoint.

In [ ]:
response = requests.get(f"{BASE_URL}/healthz")
print(f"Status code: {response.status_code}")
print(f"Response: {response.json()}")

## Generate a Simulated Trajectory

We'll use the CLI to generate a simulated trajectory. We can do this by calling the CLI directly from the notebook.

In [ ]:
import subprocess
import sys

# Generate a trajectory with 20 points
result = subprocess.run(
    [sys.executable, "-m", "qmag_nav.cli", "simulate", "--steps", "20"],
    capture_output=True,
    text=True,
    check=True,
)

# Parse the trajectory
trajectory = json.loads(result.stdout)

# Convert to DataFrame for easier manipulation
trajectory_df = pd.DataFrame(trajectory)
trajectory_df.head()

## Visualize the Simulated Trajectory

Let's plot the simulated trajectory to see what it looks like.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(trajectory_df['lon'], trajectory_df['lat'], 'o-', label='Simulated Trajectory')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Simulated Trajectory')
plt.grid(True)
plt.legend()
plt.show()

## Send Measurements to the API

Now, let's send each point in the trajectory to the API and collect the estimated positions.

In [ ]:
# List to store API responses
estimates = []

# Send each point to the API
for _, point in trajectory_df.iterrows():
    payload = {
        "lat": point['lat'],
        "lon": point['lon']
    }
    
    response = requests.post(f"{BASE_URL}/estimate", json=payload)
    
    if response.status_code == 200:
        estimates.append(response.json())
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Convert to DataFrame
estimates_df = pd.DataFrame(estimates)
estimates_df.head()

## Compare Measurements and Estimates

Let's compare the original measurements with the estimated positions.

In [ ]:
plt.figure(figsize=(12, 8))

# Plot measurements
plt.plot(trajectory_df['lon'], trajectory_df['lat'], 'o-', label='Measurements', alpha=0.7)

# Plot estimates
plt.plot(estimates_df['lon'], estimates_df['lat'], 's-', label='Estimates', alpha=0.7)

# Connect measurements to estimates with lines
for i in range(len(trajectory_df)):
    plt.plot([trajectory_df['lon'].iloc[i], estimates_df['lon'].iloc[i]],
             [trajectory_df['lat'].iloc[i], estimates_df['lat'].iloc[i]],
             'k--', alpha=0.3)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Measurements vs Estimates')
plt.grid(True)
plt.legend()
plt.show()

## Analyze Estimation Quality

Let's analyze the quality of the estimates by calculating the error between the measurements and estimates.

In [ ]:
# Calculate errors
errors = []
for i in range(len(trajectory_df)):
    # Simple Euclidean distance (not geodesic)
    error = np.sqrt(
        (trajectory_df['lat'].iloc[i] - estimates_df['lat'].iloc[i])**2 +
        (trajectory_df['lon'].iloc[i] - estimates_df['lon'].iloc[i])**2
    )
    errors.append(error)

# Plot errors over time
plt.figure(figsize=(10, 6))
plt.plot(errors, 'o-')
plt.xlabel('Measurement Index')
plt.ylabel('Error (degrees)')
plt.title('Estimation Error Over Time')
plt.grid(True)
plt.show()

# Print summary statistics
print(f"Mean error: {np.mean(errors):.6f} degrees")
print(f"Max error: {np.max(errors):.6f} degrees")
print(f"Min error: {np.min(errors):.6f} degrees")
print(f"Standard deviation: {np.std(errors):.6f} degrees")

## Conclusion

In this notebook, we've demonstrated how to:

1. Generate a simulated trajectory using the CLI
2. Send measurements to the Quantum Magnetic Navigation API
3. Visualize and analyze the estimated positions

The API provides a simple interface for position estimation based on magnetic field measurements. In a real-world scenario, you would use actual magnetometer readings instead of simulated positions.